In [3]:
import findspark
findspark.init()

In [3]:
!python -m pip install findspark

In [4]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext(master="local", appName="test")
spark = SparkSession(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=local) created by __init__ at <ipython-input-2-dace96b18753>:3 

In [ ]:
import os
dir_name="/home/sarbajit/Documents/Programs/spark/Data"

In [ ]:
from pyspark.sql.types import StructField, StructType, FloatType, StringType 
iris_schema = StructType([
    StructField("sepal_length", FloatType()),
    StructField("sepal_width", FloatType()),
    StructField("petal_length", FloatType()),
    StructField("petal_width", FloatType()),
    StructField('species', StringType())
])
iris = spark.read.csv(path=os.path.join(dir_name, "iris.csv"), schema=iris_schema)

iris.show()

In [ ]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)


In [ ]:
iris.createOrReplaceTempView("iris_temp_table")
sqlContext.sql("select sepal_length, petal_length from iris_temp_table").show()
# sqlContext.dropTempTable("iris_temp_table")

In [ ]:
# sqlContext.tableNames()
sqlContext.tables().show()

In [ ]:
iris_rdd = sc.textFile(os.path.join(dir_name, "iris.csv"))

iris_rdd_split = iris_rdd.map(lambda line : line.split(",")).map(lambda col : (col[0], col[2]))
iris_rdd_split.take(10)

In [ ]:
# python object to rdd
objToRdd =sc.parallelize([
    ('a', [10,20,30]), ('b',[100,200,300]), ('m', ['a','b','m'])
])
value
# print(objToRdd.collect())
df = spark.createDataFrame(objToRdd)
df.show()
print(sc.uiWebUrl)

In [ ]:
# Hive introduction
sqlContext.sql("drop database db")

sqlContext.sql("create database db")
sqlContext.sql("use db")

from pyspark.sql import DataFrameWriter
dfw = DataFrameWriter(iris)
# sqlContext.sql("drop table iris")
dfw.saveAsTable(name="iris", mode="overwrite")

In [ ]:
sqlContext.tables().show()

In [ ]:
# UDF 
from pyspark.sql.functions import col, udf, collect_list
from pyspark.sql.types import FloatType

def makeItUpperCase(value):
    # print(value)
    return value.upper()

doUpper = udf(makeItUpperCase, StringType())

# iris.select(doUpper(iris.species)).show()
# iris.withColumn("species", doUpper(iris.species)).show()
# iris.show()

df = iris.groupBy(iris.species).agg(collect_list(iris.sepal_length).alias("agg_sepal_length"),collect_list(iris.sepal_width).alias("agg_sepal_width"))
# print(type(df))

getSum = udf(lambda var : sum(var), returnType=FloatType())

agg_iris = df.select(df.species,getSum(df.agg_sepal_length).alias("sum of Sepal length"), getSum(df.agg_sepal_width).alias("sum_of_Sepal_width"))

agg_iris.show()
agg_iris.select(col("sum of Sepal length")).show()

In [ ]:
spark.udf.register("doUpperCase", makeItUpperCase, StringType())

sqlContext.sql("select doUpperCase(species) from iris").show()

In [ ]:
sc.uiWebUrl

In [ ]:
# Caching and Persisting 

In [ ]:
# iris.rdd.setName("iris")
df = sc.textFile("./iris.csv")
print(df)
# df.take(10)

In [ ]:

sc.parallelize((('1',),('1'),('11',1),('11'))).countByKey().items()
# sc.parallelize((('a',1),('b',2),('a',3),('a',4))).foldByKey(0,add).items()


In [ ]:
from pyspark.sql.types import IntegerType
iris.select(int(iris.sepal_length)).show()

In [ ]:
data = spark.read.format('csv').option("delimiter", "|").option('mode',"FAILFAST").load(os.path.join(dir_name,'data.txt'))
data.show()
print(data.count())
data.write.mode("Overwrite").format("parquet").option('path',os.path.join(dir_name,'out.parquet')).save()

In [ ]:
spark.read.format("parquet").load(os.path.join(dir_name,'out.parquet')).show()

In [5]:
spark.stop()